In [1]:
import torch
import time
import pprint
from llama_index.core import SimpleDirectoryReader, Settings, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.core.node_parser import SentenceSplitter
import faiss
from llama_index.vector_stores.faiss import FaissVectorStore
from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

c:\Users\ThreeThree\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ThreeThree\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\_internal\_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [3]:
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.float16, low_cpu_mem_usage=True)

tokenizer.save_pretrained("tinyllama-tokenizer")
model.save_pretrained("tinyllama-model", max_shard_size="1000MB")

In [4]:
reader = SimpleDirectoryReader(
    input_files=["Z:/fasdasd/LIDASAN_First-Thinkpiece.pdf"]
)
documents = reader.load_data()

print('Number of pages:', len(documents))
print(documents)

Number of pages: 4
[Document(id_='60a3d15b-0ee9-4032-85e9-57202b59537f', embedding=None, metadata={'page_label': '1', 'file_name': 'LIDASAN_First-Thinkpiece.pdf', 'file_path': 'Z:\\fasdasd\\LIDASAN_First-Thinkpiece.pdf', 'file_type': 'application/pdf', 'file_size': 139797, 'creation_date': '2024-05-04', 'last_modified_date': '2024-05-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Dividing a Divided Division  \n Independence was a foreign word, thought only to be attainable by the Filipinos \nof the past , first through diplomatic relations to violent revolutions. From the Spanish \nto the Americans, the Filipinos’  identity was built around  resist ing the tightening of their \ncollars , but holding  on to their leash in the event of any adverse e

In [5]:
# Initialize the parser
parser = SentenceSplitter.from_defaults(chunk_size=512, chunk_overlap=20)

# Parse documents into nodes
nodes = parser.get_nodes_from_documents(documents)
print(f"Number of nodes created: {len(nodes)}")
pprint.pprint([nodes[i] for i in range(3)])

Number of nodes created: 7
[TextNode(id_='723180e3-f356-417d-973b-fa044447e110', embedding=None, metadata={'page_label': '1', 'file_name': 'LIDASAN_First-Thinkpiece.pdf', 'file_path': 'Z:\\fasdasd\\LIDASAN_First-Thinkpiece.pdf', 'file_type': 'application/pdf', 'file_size': 139797, 'creation_date': '2024-05-04', 'last_modified_date': '2024-05-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='60a3d15b-0ee9-4032-85e9-57202b59537f', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'LIDASAN_First-Thinkpiece.pdf', 'file_path': 'Z:\\fasdasd\\LIDASAN_First-Thinkpiece.pdf', 'file_type': 'application/pdf', 'file_size': 139797, 'creation_date': '2024-05-04', 'last_modified_date': '20

In [6]:
#Create a Faiss index. 768 is the dimensionality of the embeddings generated by sentence-transformers
faiss_index = faiss.IndexFlatL2(768)

#Load the embedding model
Settings.embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="stsb-distilbert-base")
)

#Create a vector storage and its context
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

#Add the embeddings to the index
index = VectorStoreIndex(nodes, storage_context=storage_context)

# save index to disk. Will be stored in ./storage by default
index.storage_context.persist()

In [7]:
Settings.llm = HuggingFaceLLM(
    context_window=2048,
    max_new_tokens=512,
    generate_kwargs={"temperature": 0.1, "do_sample": False},
    tokenizer_name="tinyllama-tokenizer",
    model_name="tinyllama-model",
    tokenizer_kwargs={"max_length": 2048},
    model_kwargs={"torch_dtype": torch.float16}
)

# Settings.llm = HuggingFaceLLM(
#     context_window=2048,
#     max_new_tokens=512,
#     generate_kwargs={"temperature": 0.1, "do_sample": False},
#     tokenizer_name="HuggingFaceH4/zephyr-7b-beta",
#     model_name="HuggingFaceH4/zephyr-7b-beta",
#     tokenizer_kwargs={"max_length": 2048},
#     model_kwargs={"torch_dtype": torch.float16}
# )
     

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  6.21it/s]
The model `tinyllama-model` and tokenizer `tinyllama-tokenizer` are different, please ensure that they are compatible.


In [8]:
stored_index = load_index_from_storage(storage_context)

query_engine = stored_index.as_query_engine()
prompt="If the philippines would actually flourish if we employed the same government style as United States of America"

In [9]:
t0=time.time()
response = query_engine.query(prompt)
print(f"Time: {time.time()-t0}")

c:\Users\ThreeThree\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from IPython.display import display, HTML


pd.set_option("display.max_colwidth", None)


def pretty_print(df):
    return display(HTML(df.to_html().replace("\n", "")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))


print(response.response)

nodes= response.source_nodes
visualize_retrieved_nodes(nodes)


The Internet of Things (IoT) refers to the network of physical devices, vehicles, home appliances, and other items embedded with electronics, software, sensors, actuators, and wireless networking technologies that enable these objects to connect and exchange data with each other and the outside world. IoT is transforming the way we live, work, and interact with the world around us. It is a rapidly growing technology that is expected to have a significant impact on various industries, including manufacturing, transportation, healthcare, and energy. The IoT is also transforming the way we think about security, privacy, and data management. IoT devices are becoming increasingly connected, and this connectivity is creating new opportunities for cybersecurity threats. However, IoT also presents significant opportunities for innovation and economic growth.


,Score,Text
0,391.681335,"“…creation of a nationally integrated economy rather than on the creation of \nmushroom states based on ethno -linguistic sentiments for sharing the national \nwealth.” The Filipinos’ affinity for their place of origin rather than their state has \npersisted way before the American Occupation as a result of intolerance for diversity \nand understandable distrust of the foreign visitors. In contemporary times, this affinity \nremains to be the chosen tool to pursue charter change, citing financial distribution \nand self -management as the selling poi nt of acceptance. To combat the weak nation -\nstate relationship in the Philippines, a proposed solution was Federalism through the \nact of decentralization. Decentr alization allows the distribution of power amongst \nvarious local leaders, thereby building rapport and a tangible relationship between the \ngovernment and the people through “administrative de -concentration and political \ndevolution” (Brillantes & Moscare, 2002) On paper, it is convincing, evidently showing \npart of the weakness in nation -building which is effective communication and \nrepresentation. To its benefit, federal system can solve that to an extent since local \nstate government bodies are provided their own respective amounts of political power \nby the national government. In contrast, the legitimacy of federalism as a palpable \nnation -building tool remains questioned. \n The proposition of a federal system to improve the sociopolitical situation in the \nPhilippines may be enticing at best, but it is of absolute importance that we look \ndeeper into the notions of such a major change. Reasons presented by countless \npolitical figures in favor of the change is only a façade to the reality that will become \nof our country if a federal system were adopted now or even during the American \noccupation. Nation -building through separation and distribution of powers can only \nwork if the n ation has already built a connection with their state that encompasses \ngeographical distance and cultural diversity to be recognized as one identity. \nMoreover, the enduring issue of internal discrimination among Filipinos by placing a \nderogatory emphasis o n their place of origin is pushing them away from us in what \nshould have been a unified nation."
1,404.326233,"REFERENCES \nBrillantes, A., & Moscare , D. (2002). Decentralization and Federalism in the \nPhilippines: Lessons from Global Community . \nFoster, K. (2001). Regionalism on Purpose . \nhttps://www.lincolninst.edu/sites/default/files/pubfiles/regionalism -on-purpose -\nfull.pdf \nLlanera, T. (2017). Ethnocentrism: Lessons from Richard Rorty to Randy David. \nPhilippine Sociological Review , 65, 133 –149. \nhttps://www.jstor.org/stable/45014312 \nOluniyi, A. E. (2014). REGIONALISM, IDEOLOGY CRISES, PARTY AFFILIATION \nAND FUTURE OF DEMOCRACY IN NIGERIA. Afro Asian Journal of Social \nSciences , 5(5.1), 1 –20. \nhttp://www.onlineresearchjournals.com/aajoss/art/125.pdf \nSöderberg Kovacs, M., Höglund, K., & Jiménez, M. (2021). Autonomous Peace? \nThe Bangsamoro Region in the Philippines Beyond the 2014 Agreement. \nJournal of Peacebuilding & Development , 16(1), 55 –69. \nhttps://doi.org/10.1177/1542316620987556"
